# Démonstration manipulation des données extraites de l'API

Ce notebook présente comment les données pourraient être affichées et comment elles pourraient être utiles pour construite un modèle prédictif des emplacements et vélos disponibles sur le réseau Vélo Toulouse

In [13]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [14]:
df = pd.read_csv('Extract_data_test.csv')
df

,Unnamed: 0,datetime,number,name,address,bike_stands,available_bike_stands,available_bikes
0,146,2025-10-21 13:01:32+02:00,402,00402 - GRYNFOGEL - GAILLARDIE,"Face 130, Chemin de Gaillardie",16,16,0
1,280,2025-10-21 13:01:37+02:00,261,00261 - VAN DYCK - RAPHAËL,DEVANT 6 RUE VAN DICK / FACE RUE RAPHAEL,18,12,0
2,396,2025-10-21 13:01:38+02:00,203,00203 - ST-EXUPERY - PETIT PRINCE,2 AV ANTOINE DE ST EXUPERY,20,10,10
3,142,2025-10-21 13:01:41+02:00,252,00252 - CATELLAN - CAZENEUVE,RUE MARIE CLAIRE DE CATELLAN (10M AVANT LE PON...,18,4,12
4,208,2025-10-21 13:01:50+02:00,312,00312 - BASE DE LOISIRS SESQUIERES,Lac de Sesquières - Devant l'entrée Wam Park,13,9,4
...,...,...,...,...,...,...,...,...
7758,77,2025-10-21 14:10:35+02:00,175,00175 - BORDEROUGE - MÉTRO,FACE 52 AV BOURGES MAUNOURY,30,18,11
7759,351,2025-10-21 14:10:35+02:00,317,00317 - GAUSSEN - CROIX-BÉNITE,"Face, 22 - 24 boulevard Henri Gaussen",13,10,3
7760,64,2025-10-21 14:10:35+02:00,3004,03004 - MONTEL - LANGUEDOC,"Devant 17, boulevard Eugène Montel - Tournefeu...",16,5,11
7761,415,2025-10-21 14:10:36+02:00,14,00014 - PERIGORD,26 RUE DE PERIGORD,17,11,5


On sélectionne deux stations à étudier: celle de l'ISAE-SUPAERO et celle de la Place Saint Pierre.

In [16]:
# --- Liste des stations ---
stations = {
    224: "BELIN - SUPAERO",
    28: "SAINT-PIERRE",
}

On trace les séries temporelles représentant l'évolution de la disponibilité des vélos et emplacements disponibles dans ces stations. Les données ont été collectées le 21 Octobre 2025, de 13h10 à 14h10. 

In [ ]:
# --- Création du subplot (2 lignes, 1 colonne) ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=[f"{name} (Station {num})" for num, name in stations.items()],
    vertical_spacing=0.1
)

# --- Couleurs cohérentes ---
color_bornes = px.colors.qualitative.Plotly[0]  # bleu
color_velos = px.colors.qualitative.Plotly[1]   # orange

# --- Calcul de l'extension de l'axe X ---
df["datetime"] = pd.to_datetime(df["datetime"])
x_min = df['datetime'].min()
x_max = df['datetime'].max()
x_extended = x_max
# x_extended = x_min + 2 * (x_max - x_min)  # moitié droite vide

for i, (num, name) in enumerate(stations.items(), start=1):
    df_station = df[df["number"] == num]

    # Bornes libres
    fig.add_trace(
        go.Scatter(
            x=df_station["datetime"],
            y=df_station["available_bike_stands"],
            mode="lines+markers",
            name="Bornes libres" if i == 1 else None,  # une seule légende
            line=dict(color=color_bornes, width=3),
            marker=dict(size=5, symbol="circle"),
        ),
        row=i, col=1
    )

    # Vélos disponibles
    fig.add_trace(
        go.Scatter(
            x=df_station["datetime"],
            y=df_station["available_bikes"],
            mode="lines+markers",
            name="Vélos disponibles" if i == 1 else None,  # une seule légende
            line=dict(color=color_velos, width=3, dash="dot"),
            marker=dict(size=5, symbol="square"),
        ),
        row=i, col=1
    )

for i in range(1, 3):
    fig.add_trace(
        go.Scatter(
            x=[x_extended],
            y=[0],
            mode="markers",
            marker=dict(opacity=0),
            showlegend=False
        ),
        row=i, col=1
    )

fig.update_layout(
    height=1000,
    title=dict(
        text="Évolution du nombre de vélos et bornes libres — Toulouse",
        x=0.5,
        xanchor="center",
        font=dict(size=22)
    ),
    xaxis=dict(
        title="Date et heure",
        range=[x_min, x_extended],
        showgrid=True,
        gridcolor="rgba(220,220,220,0.3)",
        tickformat="%d/%m %H:%M"
    ),
    xaxis2=dict(
        title="Date et heure",
        range=[x_min, x_extended],
        showgrid=True,
        gridcolor="rgba(220,220,220,0.3)",
        tickformat="%d/%m %H:%M"
    ),
    yaxis=dict(title="Nombre", showgrid=True, gridcolor="rgba(220,220,220,0.3)"),
    yaxis2=dict(title="Nombre", showgrid=True, gridcolor="rgba(220,220,220,0.3)"),
    plot_bgcolor="white",
    hovermode="x unified",
    legend=dict(
        title="Légende",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        font=dict(size=12)
    ),
    font=dict(family="Arial", size=13)
)

fig.update_annotations(font_size=16)
fig.show()

Le but est maintenant d'avoir un modèle de forecasting qui "prolonge" ces courbes